<a href="https://colab.research.google.com/github/Ala-Mohamed/ASD-diagnosis-using-fMRI-A-computer-aided-approach-/blob/main/DL_Approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyinform
!pip install shap
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.2/131.2 kB 8.2 MB/s eta 0:00:00


In [ ]:
import os
import glob
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from pyinform import transfer_entropy
import shap
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")



In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import glob
import numpy as np

# === 1. Load CSV Matrices ===
asd_folder = r'/content/drive/MyDrive/matrices-bold-time-series/ASD'
td_folder = r'/content/drive/MyDrive/matrices-bold-time-series/HC'

asd_files = sorted(glob.glob(os.path.join(asd_folder, "*.csv")))
td_files = sorted(glob.glob(os.path.join(td_folder, "*.csv")))

print(f"ASD folder exists: {os.path.exists(asd_folder)}")
print(f"TD folder exists: {os.path.exists(td_folder)}")


# Print the number of files found
print(f"Number of ASD files found: {len(asd_files)}")
print(f"ASD files: {asd_files}")
print(f"Number of TD files found: {len(td_files)}")
print(f"TD files: {td_files}")

# Check if files were found
if not asd_files or not td_files:
    raise FileNotFoundError("No CSV files found in one or both folders. Please check the folder paths and file extensions.")

# Load the matrices with error handling
asd_matrices = []
for f in asd_files:
    try:
        matrix = np.genfromtxt(f, delimiter=',')
        asd_matrices.append(matrix)
    except Exception as e:
        print(f"Error reading file {f}: {e}")

td_matrices = []
for f in td_files:
    try:
        matrix = np.genfromtxt(f, delimiter=',')
        td_matrices.append(matrix)
    except Exception as e:
        print(f"Error reading file {f}: {e}")

# Print the number of matrices loaded
print(f"Number of ASD matrices loaded: {len(asd_matrices)}")
print(f"Number of TD matrices loaded: {len(td_matrices)}")

np.random.seed(42)
n_subjects = 25
if n_subjects == 0:
    raise ValueError("No valid matrices found to process.")

# Subset the data
asd_subset_indices = np.random.choice(len(asd_matrices), size=n_subjects, replace=False)
td_subset_indices = np.random.choice(len(td_matrices), size=n_subjects, replace=False)

asd_subset = [asd_matrices[i] for i in asd_subset_indices]
td_subset = [td_matrices[i] for i in td_subset_indices]

# Combine ASD and TD data and create labels
all_matrices = asd_subset + td_subset
labels = np.array([1] * n_subjects + [0] * n_subjects)  # 1 for ASD, 0 for TD

# Define sliding window parameters
window_size = 20  # Window size in time points
step_size = 2     # Step size for sliding window
n_rois = 122      # Number of ROIs in BASC atlas

# Directory to save transfer entropy matrices
output_dir = '/content/transfer_entropy_matrices'
os.makedirs(output_dir, exist_ok=True)

ASD folder exists: True
TD folder exists: True
Number of ASD files found: 172
ASD files: ['/content/drive/MyDrive/matrices-bold-time-series/ASD/ASD0.csv', '/content/drive/MyDrive/matrices-bold-time-series/ASD/ASD1.csv', '/content/drive/MyDrive/matrices-bold-time-series/ASD/ASD10.csv', '/content/drive/MyDrive/matrices-bold-time-series/ASD/ASD100.csv', '/content/drive/MyDrive/matrices-bold-time-series/ASD/ASD101.csv', '/content/drive/MyDrive/matrices-bold-time-series/ASD/ASD102.csv', '/content/drive/MyDrive/matrices-bold-time-series/ASD/ASD103.csv', '/content/drive/MyDrive/matrices-bold-time-series/ASD/ASD104.csv', '/content/drive/MyDrive/matrices-bold-time-series/ASD/ASD105.csv', '/content/drive/MyDrive/matrices-bold-time-series/ASD/ASD106.csv', '/content/drive/MyDrive/matrices-bold-time-series/ASD/ASD107.csv', '/content/drive/MyDrive/matrices-bold-time-series/ASD/ASD108.csv', '/content/drive/MyDrive/matrices-bold-time-series/ASD/ASD109.csv', '/content/drive/MyDrive/matrices-bold-time-s

In [ ]:
# Function to discretize time-series data for pyinform
def discretize_time_series(time_series, n_bins=10):
    # Bin the time-series into discrete states
    min_val, max_val = np.nanmin(time_series), np.nanmax(time_series)
    bins = np.linspace(min_val, max_val, n_bins + 1)
    discretized = np.digitize(time_series, bins, right=True)
    discretized = np.clip(discretized, 0, n_bins - 1)  # Ensure values are within [0, n_bins-1]
    return discretized

# Function to compute transfer entropy matrix for a time-series window using pyinform
def compute_transfer_entropy(time_series, delay=1, k=1):
    n_rois = time_series.shape[1]
    te_matrix = np.zeros((n_rois, n_rois))
    # Discretize the time-series
    discretized = np.apply_along_axis(discretize_time_series, 0, time_series, n_bins=10)
    for i in range(n_rois):
        for j in range(i + 1, n_rois):
            target = discretized[:, i].astype(int)
            # Shift source series by delay
            source = np.roll(discretized[:, j].astype(int), -delay)
            # Trim the rolled array to match the original length (remove padded values)
            source = source[:-delay] if delay > 0 else source
            target = target[:-delay] if delay > 0 else target
            if len(source) == len(target) and len(source) >= k + delay + 1:  # Ensure enough data
                te = transfer_entropy(source, target, k=k)
                te_matrix[i, j] = te if not np.isnan(te) else 0  # Handle NaN with 0
                te_matrix[j, i] = te  # Symmetric for simplicity
    return te_matrix

# Function to apply sliding window and compute transfer entropy matrices
def compute_sliding_te_matrices(time_series, window_size, step_size):
    n_time_points = time_series.shape[0]
    te_matrices = []
    for start in range(0, n_time_points - window_size + 1, step_size):
        window = time_series[start:start + window_size]
        te_matrix = compute_transfer_entropy(window, delay=1, k=1)
        te_matrices.append(te_matrix)
    return np.array(te_matrices)

# Compute and save transfer entropy matrices for all subjects
all_te_matrices = []
for idx, matrix in enumerate(all_matrices):
    output_file = os.path.join(output_dir, f'subject_{idx}_te_matrices.npy')
    if os.path.exists(output_file):
        te_matrices = np.load(output_file)
    else:
        # Handle NaNs in the time-series before computing TE
        imputer = SimpleImputer(strategy='mean')
        matrix_imputed = imputer.fit_transform(matrix)
        te_matrices = compute_sliding_te_matrices(matrix_imputed, window_size, step_size)
        np.save(output_file, te_matrices)
    all_te_matrices.append(te_matrices)


In [ ]:

# Ensure all subjects have the same number of windows
n_windows = min([m.shape[0] for m in all_te_matrices])
all_te_matrices = [m[:n_windows] for m in all_te_matrices]

# Flatten each TE matrix into a feature vector and ensure consistent shape
n_features = n_rois * (n_rois - 1) // 2
X = np.array([matrix[:n_windows, np.triu_indices(n_rois, k=1)] for matrix in all_te_matrices], dtype=np.float32)
# Shape of X should be (n_subjects, n_windows, n_features)

# Reshape for LSTM: (n_subjects * n_windows, sequence_length, n_features)
sequence_length = n_windows
X_reshaped = X.reshape(n_subjects * n_windows, sequence_length, n_features)
y_reshaped = np.tile(labels, n_windows)  # Tile labels to match the total number of windows

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_reshaped, y_reshaped, test_size=0.2, random_state=42)

# Apply PCA to reduce dimensionality before LSTM to manage memory
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=50))
])

# Reshape for PCA, apply, and reshape back
X_train_flat = X_train.reshape(-1, n_features)
X_test_flat = X_test.reshape(-1, n_features)
X_train_reduced = pipeline.fit_transform(X_train_flat).reshape(X_train.shape[0], sequence_length, 50)
X_test_reduced = pipeline.transform(X_test_flat).reshape(X_test.shape[0], sequence_length, 50)


In [ ]:
import os
import glob
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.utils.class_weight import compute_class_weight
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking
from tensorflow.keras.callbacks import EarlyStopping
from pyinform import transfer_entropy
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor
import warnings
warnings.filterwarnings("ignore")

# ======================
# 1. Configuration
# ======================
config = {
    'asd_folder': '/content/drive/My Drive/matrices-bold-time-series/ASD',
    'td_folder': '/content/drive/My Drive/matrices-bold-time-series/HC',
    'window_size': 20,
    'step_size': 5,
    'n_rois': 122,
    'te_delay': 1,
    'te_k': 1,
    'n_bins': 10,
    'pca_components': 50,
    'lstm_units': 128,
    'dropout_rate': 0.3,
    'batch_size': 32,
    'epochs': 100,
    'test_size': 0.2,
    'random_state': 42
}

# ======================
# 2. Data Loading
# ======================
def load_data(folder):
    files = sorted(glob.glob(os.path.join(folder, "*.csv")))
    matrices = []
    valid_files = []

    for f in files:
        try:
            matrix = pd.read_csv(f, header=None).values
            if matrix.shape[1] == config['n_rois']:
                matrices.append(matrix)
                valid_files.append(f)
            else:
                print(f"Invalid shape in {f}: {matrix.shape}")
        except Exception as e:
            print(f"Error reading {f}: {e}")

    print(f"Loaded {len(matrices)} valid matrices from {folder}")
    return matrices, valid_files

asd_matrices, asd_files = load_data(config['asd_folder'])
td_matrices, td_files = load_data(config['td_folder'])

# Handle class imbalance
min_samples = min(len(asd_matrices), len(td_matrices))
asd_matrices = asd_matrices[:min_samples]
td_matrices = td_matrices[:min_samples]

# Create labels (1 for ASD, 0 for TD)
labels = np.concatenate([np.ones(len(asd_matrices)), np.zeros(len(td_matrices))])
all_matrices = asd_matrices + td_matrices

# Subject-level split
subj_ids = np.arange(len(all_matrices))
train_idx, test_idx, y_train, y_test = train_test_split(
    subj_ids, labels, test_size=config['test_size'],
    stratify=labels, random_state=config['random_state']
)

# ======================
# 3. Transfer Entropy Calculation (Fixed)
# ======================
def discretize(series, n_bins=10):
    """Discretize time series into bins"""
    min_val, max_val = np.nanmin(series), np.nanmax(series)
    if min_val == max_val:
        return np.zeros_like(series, dtype=int)
    bins = np.linspace(min_val, max_val, n_bins + 1)
    return np.digitize(series, bins) - 1

def calculate_te_window(window):
    """Calculate TE matrix for a single window"""
    n_rois = window.shape[1]
    te_matrix = np.zeros((n_rois, n_rois))

    # Discretize entire window first
    discretized = np.apply_along_axis(discretize, 0, window, config['n_bins'])

    for target in range(n_rois):
        target_series = discretized[:, target]

        for source in range(n_rois):
            if source == target:
                continue

            source_series = discretized[:, source]

            # Calculate TE with error handling
            try:
                te_val = transfer_entropy(
                    source_series, target_series,
                    k=config['te_k']
                )
                te_matrix[target, source] = te_val if not np.isnan(te_val) else 0
            except:
                te_matrix[target, source] = 0

    return te_matrix

def process_subject(subject_id):
    """Process a single subject's time series"""
    matrix = all_matrices[subject_id]
    imputer = SimpleImputer(strategy='mean')
    matrix = imputer.fit_transform(matrix)

    te_matrices = []
    n_timepoints = matrix.shape[0]

    for start in range(0, n_timepoints - config['window_size'], config['step_size']):
        window = matrix[start:start + config['window_size']]
        te_matrix = calculate_te_window(window)
        te_matrices.append(te_matrix)

    return np.array(te_matrices)

# Parallel processing of subjects
print("Computing transfer entropy matrices...")
with ProcessPoolExecutor() as executor:
    all_te_results = list(executor.map(process_subject, range(len(all_matrices))))

# ======================
# 4. Feature Engineering
# ======================
def extract_graph_metrics(te_matrix):
    """Extract network metrics from TE matrix"""
    # Threshold to create adjacency matrix
    threshold = np.percentile(te_matrix, 90)
    adj_matrix = (te_matrix > threshold).astype(float)

    # Calculate graph metrics
    out_degree = adj_matrix.sum(axis=1)
    in_degree = adj_matrix.sum(axis=0)

    return np.array([
        out_degree.mean(), out_degree.std(),
        in_degree.mean(), in_degree.std(),
        adj_matrix.mean(),
        np.median(te_matrix),
        np.mean(te_matrix > 0)
    ])

# Extract features for all subjects
X = []
for subject_te in all_te_results:
    subject_features = [extract_graph_metrics(te) for te in subject_te]
    X.append(subject_features)

# Handle variable sequence lengths
max_windows = max(len(subj) for subj in X)
X_padded = np.array([
    np.vstack([
        subj,
        np.zeros((max_windows - len(subj), len(X[0][0]))
    ]) for subj in X
])

# Create mask for variable lengths
mask = np.array([
    [1] * len(subj) + [0] * (max_windows - len(subj))
    for subj in X
])

# Split data based on subject-level split
X_train = X_padded[train_idx]
y_train_full = labels[train_idx]
mask_train = mask[train_idx]

X_test = X_padded[test_idx]
y_test_full = labels[test_idx]
mask_test = mask[test_idx]

# ======================
# 5. Model Architecture
# ======================
# Preprocessing pipeline
preprocessor = make_pipeline(
    SimpleImputer(strategy='mean'),
    StandardScaler(),
    PCA(n_components=config['pca_components'])
)

# Apply preprocessing
X_train_preprocessed = preprocessor.fit_transform(
    X_train.reshape(-1, X_train.shape[-1])
).reshape(X_train.shape[0], X_train.shape[1], config['pca_components'])

X_test_preprocessed = preprocessor.transform(
    X_test.reshape(-1, X_test.shape[-1])
).reshape(X_test.shape[0], X_test.shape[1], config['pca_components'])

# Handle class imbalance
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train_full),
    y=y_train_full
)
class_weight_dict = dict(enumerate(class_weights))

# Build LSTM model
model = Sequential([
    Masking(mask_value=0., input_shape=(None, config['pca_components'])),
    LSTM(config['lstm_units'], return_sequences=False),
    Dropout(config['dropout_rate']),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(name='auc')]
)

# Train model
history = model.fit(
    X_train_preprocessed, y_train_full,
    sample_weight=mask_train.mean(axis=1),  # Weight by valid windows
    class_weight=class_weight_dict,
    epochs=config['epochs'],
    batch_size=config['batch_size'],
    validation_split=0.1,
    callbacks=[EarlyStopping(patience=10, restore_best_weights=True)],
    verbose=1
)

# ======================
# 6. Evaluation
# ======================
# Evaluate on test set
test_loss, test_acc, test_auc = model.evaluate(
    X_test_preprocessed, y_test_full, verbose=0
)

y_pred = model.predict(X_test_preprocessed).flatten()
y_pred_binary = (y_pred > 0.5).astype(int)

precision = precision_score(y_test_full, y_pred_binary)
recall = recall_score(y_test_full, y_pred_binary)

print("\n" + "="*50)
print("Final Evaluation Metrics:")
print(f"Test Accuracy: {test_acc:.4f}")
print(f"Test AUC: {test_auc:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print("="*50)

# ======================
# 7. Visualization
# ======================
# Plot training history
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.tight_layout()
plt.show()

# Plot ROC curve
from sklearn.metrics import roc_curve, auc
fpr, tpr, _ = roc_curve(y_test_full, y_pred)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2,
         label=f'ROC curve (area = {roc_auc:.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()